# 네이버 블로그로 파급력을 측정합니다.

In [ ]:
import urllib.request
import json
import pandas as pd
import re
# 글에서 해당 키워드 찾기
from konlpy.tag import Kkma
# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# 트리맵
import squarify

In [ ]:
# 파일 가져오기
naver_total = pd.read_csv("./data/naver_blog/total_power.csv")
# 형 변환
naver_total["oldestDate"] = pd.to_datetime(naver_total["oldestDate"]).apply(pd.Timestamp)
naver_total["referenceDate"] = pd.to_datetime(naver_total["referenceDate"]).apply(pd.Timestamp)
naver_total["mostRecentDate"] = pd.to_datetime(naver_total["mostRecentDate"]).apply(pd.Timestamp)
# power3열 추가
# (기준일 후 게시물 개수 / [(가장 마지막 게시물 날짜- 기준일) / (마지막날짜 - 오래된 날짜) * 100] - 기준일 전 게시물 개수 / [(기준일 - 게시물 중 가장 오래된 날짜) / (마지막날짜 - 오래된 날짜) * 100]
# [기준일 후 게시물 개수 / (가장 마지막 게시물 날짜- 기준일) - 기준일 전 게시물 개수 / (기준일 - 게시물 중 가장 오래된 날짜)]* 100 *(마지막날짜 - 오래된 날짜)
naver_total["power3"] = ( naver_total["reference_after"] / pd.to_numeric((naver_total["mostRecentDate"] - naver_total["referenceDate"]).dt.days, downcast='integer') - naver_total["reference_before"] / pd.to_numeric((naver_total["referenceDate"] - naver_total["oldestDate"]).dt.days, downcast='integer')) * pd.to_numeric((naver_total["mostRecentDate"] - naver_total["oldestDate"]).dt.days, downcast='integer') / 100
# reliability2열 추가
naver_total["reliability2"] = naver_total["resource_count"] / (naver_total["reference_after"] - naver_total["resource_count"]) * 100
# reliability열 추가
naver_total["reliability"] = naver_total["resource_count"] / naver_total["reference_after"] * 100
# 결측치 처리 
naver_total['power3'].fillna(0, inplace=True)
naver_total['reliability2'].fillna(100, inplace=True)
naver_total['reliability'].fillna(0, inplace=True)
# 그룹열 만들기
연예인 = ["이영자", "백종원", "성시경", "신동엽", "최자", "비(정지훈)", "문세윤", "김준현"]
유튜버 = ["더들리", "츄릅켠"]
방송프로그램 = ["수요미식회", "생생정보통", "전참시", "생활의 달인", "6시 내 고향", "골목식당", "생방송 오늘 저녁", "모닝 와이드", "맛있는 녀석들", "생방송 투데이"]
groups = [연예인, 유튜버, 방송프로그램]
groups_names = ["연예인", "유튜버", "방송프로그램"]
for i, groups in enumerate(groups):
    for media in groups:
        naver_total.loc[naver_total["resource"] == media, "group"] = groups_names[i]
display(naver_total)
naver_total.to_csv("./data/naver_blog/total_power.csv", index=False)

# 시각화

In [ ]:
# 폰트 임시파일이 설치된 곳
matplotlib.get_cachedir()

In [ ]:
# 폰트설정
from matplotlib import font_manager, rc
font_path = "./data/font/21582067_G마켓산스Medium_OTF.otf"   #폰트파일의 위치
font_name = font_manager.FontProperties(fname=font_path).get_name() # rc 함수를 통해, rcParams에 사용되는 폰트의 정식명칭을 알 수 있음
print("폰트설정 :", font_name)
rc('font', family=font_name)
# 데이터 가져오기
data = pd.read_csv("./data/naver_blog/total_power.csv")
display(data)

## 1. 산점도 그리기 : (resource - power3) (20 - (-? - ?))

In [ ]:
# 스타일 테마 설정 (5가지: darkgrid, whitegrid, dark, white, ticks)
# sns.set_style('white')
# sns.set(rc={'figure.figsize':(11.7,8.27)})
plt.figure(figsize=(16,6))

#이산형 변수의 분포 - 데이터 분산 고려 (중복 X) 
sns.swarmplot(x="resource",      #x축 변수
              y="power3",        #y축 변수
              data=data)   #데이터셋 - 데이터프레임

# data.plot(kind='scatter', x='resource', y='power3', marker='+',
#         cmap='viridis', s=50, alpha=0.3, figsize=(20, 5))
plt.title()
plt.show()

## 2. 나무맵 : 블로그 게시물 개수(200개 아님)

In [ ]:
!pip install squarify

In [ ]:
# 나무맵 : 전수조사용
plt.figure(figsize=(10,5))
# 라벨 구하기
temp = data.copy()
#temp = temp.sort_values(by="blogTotal", ascending = False)
temp.loc[temp["blogTotal"] <= 250000,"restaurant"] = ""
squarify.plot(sizes=data["blogTotal"], label = temp["restaurant"], alpha=0.6)
plt.axis('off')
plt.savefig('./output/treemap.png')
plt.show()

In [ ]:
# 나무맵 : 항목별 5개씩
rastaurant = pd.read_excel("./output/음식점 모음.xlsx", sheet_name="Sheet1")
rastaurant = rastaurant.head(5)
temp = pd.DataFrame()
연예인 = ["이영자", "백종원", "성시경", "신동엽", "최자", "비(정지훈)", "문세윤", "김준현"]
유튜버 = ["더들리", "츄릅켠"]
방송프로그램 = ["수요미식회", "생생정보통", "전참시", "생활의 달인", "6시 내 고향", "골목식당", "생방송 오늘 저녁", "모닝 와이드", "맛있는 녀석들", "생방송 투데이"]
names = 연예인 + 유튜버 + 방송프로그램
for name in names:
    for i in range(len(rastaurant[name])):
        mask = (data["resource"] == name) & (data["restaurant"] == rastaurant[name][i])
        temp = temp.append(data.loc[mask].head(1))
# 라벨 구하기
temp.loc[temp["blogTotal"] <= 20000,"restaurant"] = ""
# 그래프 그리기
plt.figure(figsize=(10,5))
squarify.plot(sizes=temp["blogTotal"], label = temp["restaurant"], alpha=0.6)
plt.axis('off')
plt.savefig('./output/treemap2.png')
plt.show()

## 3. 막대 그래프 그리기
* file : power3(사람별 - 해당 미디어의 영향력) power3값을 평균 사람별로(방송별) 막대그래프 (x-20개 y-(-?, ?))
* (사람별 - 영향력의 크기) reliability2값을 평균 사람별로(방송별) 막대그래프 (x-20개 y-(-?, ?))
* (사람별 - 글을 작성할 확률) reliability값을 평균 사람별로(방송별) 막대그래프 (x-20개 y-(-?, ?))

In [ ]:
df1 = data.groupby("resource").mean()
display(df1)

In [ ]:
plt.figure(figsize=(16,6))
plt.title("사람별 식당 언급 횟수의 평균")
r = df1.resource_count.plot(kind='bar')
print(r)

In [ ]:
plt.figure(figsize=(20,5))

x = df1.index
y = df1.power3
plt.title("사람별 파급력(power)과 신뢰도(reliability)의 평균")
my_color = np.where(y>=0, 'skyblue', 'orange')
plt.bar(x, y, color=my_color)

for i, v in enumerate(x):
    if df1.power3[i] > 0:
        plt.text(v, y[i], round(df1.reliability[i], 2),       
                 fontsize = 9, 
                 color='black',
                 horizontalalignment='center',  
                 verticalalignment='bottom') 
    else :
        plt.text(v, y[i], round(df1.reliability[i], 2),       
                 fontsize = 9, 
                 color='black',
                 horizontalalignment='center',  
                 verticalalignment='top')
plt.ylim(-2,19)
plt.show()

<br>

## 4. 분산분석을 통하여 3개 이상의 집단 간에 차이가 존재하는지 분석하고자 함

가설1 : 미디어에 따라 파급력의 차이가 있는가?  
가설2 : 미디어에 따라 파급력을 미치는 크기에 차이가 있는가?  
가설3 : 미디어에 따라 글을 작성할 확률에 차이가 있는가?

<hr>

분산분석(미디어별 - 해당 미디어의 영향력) power값  
<p>분산분석(미디어별 - 영향력의 크기) reliability2값</p>
<p>분산분석(미디어별 - 글을 작성할 확률) reliability값</p>

#### 가설1 : 미디어에 따라 파급력의 차이가 있는가?

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

df = data
model = ols("df['power3'] ~ df['group']", df).fit()
anova_lm(model)

In [ ]:
# 집단 간의 차이가 유의할 경우, 어느 집단 간에 차이가 나타나는지 보다 구체적으로 보기 위하여 '사후 분석'을 실시
# 여기서는 봉페르니 방식의 교정과 Tukey HSD의 방법으로 사후분석
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

comp = MultiComparison(df['power3'], df['group'])
result = comp.allpairtest(scipy.stats.ttest_ind, method='bonf') # 봉페르니
print(result[0])

#투키의 HSD - Tuckey's Honestly Significant Difference = "진정으로 유의미한 차이"
from statsmodels.stats.multicomp import pairwise_tukeyhsd
hsd = pairwise_tukeyhsd(df['power3'], df['group'], alpha=0.05)
hsd.summary()

#### 가설2 : 미디어에 따라 파급력을 미치는 크기에 차이가 있는가?

In [ ]:
model = ols("df['reliability'] ~ df['group']", df).fit()
anova_lm(model)

In [ ]:
# 집단 간의 차이가 유의할 경우, 어느 집단 간에 차이가 나타나는지 보다 구체적으로 보기 위하여 '사후 분석'을 실시
# 여기서는 봉페르니 방식의 교정과 Tukey HSD의 방법으로 사후분석
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

comp = MultiComparison(df['reliability'], df['group'])
result = comp.allpairtest(scipy.stats.ttest_ind, method='bonf') # 봉페르니
print(result[0])

#투키의 HSD - Tuckey's Honestly Significant Difference = "진정으로 유의미한 차이"
from statsmodels.stats.multicomp import pairwise_tukeyhsd
hsd = pairwise_tukeyhsd(df['reliability'], df['group'], alpha=0.05)
hsd.summary()

#### 가설3 : 미디어에 따라 글을 작성할 확률에 차이가 있는가?

In [ ]:
# 집단 간의 차이가 유의할 경우, 어느 집단 간에 차이가 나타나는지 보다 구체적으로 보기 위하여 '사후 분석'을 실시
# 여기서는 봉페르니 방식의 교정과 Tukey HSD의 방법으로 사후분석
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

comp = MultiComparison(df['reliability2'], df['group'])
result = comp.allpairtest(scipy.stats.ttest_ind, method='bonf') # 봉페르니
print(result[0])

#투키의 HSD - Tuckey's Honestly Significant Difference = "진정으로 유의미한 차이"
from statsmodels.stats.multicomp import pairwise_tukeyhsd
hsd = pairwise_tukeyhsd(df['reliability2'], df['group'], alpha=0.05)
hsd.summary()

#### 분석 결과 해석
: Pr(>F)== p-value. 
<hr>  

***

##### 가설1 : 미디어에 따라 파급력의 차이가 있는가?

분산분석: p-value가 0.05보다 작으므로, 집단 간에 통계적으로 유의한 차이가 나타남
사후분석: pval의 값이 0.05보다 낮으므로, 방송프로그램과 유튜버 간의 차이가 유의하여 주효과가 나타난다.

##### 가설2 : 미디어에 따라 파급력을 미치는 크기에 차이가 있는가?

분산분석: p-value가 부동소수로 표현 될만큼 낮은 숫자이고, 0.05보다 작으므로 집단 간에 통계적으로 유의한 차이가 나타남
사후분석: pval의 값이 0.05보다 낮으므로, 방송프로그램과 연예인 간의 차이가 유의하여 주효과가 나타난다.
##### 가설3 : 미디어에 따라 글을 작성할 확률에 차이가 있는가?

분산분석: p-value가 부동소수로 표현 될만큼 낮은 숫자이고, 0.05보다 작으므로 집단 간에 통계적으로 유의한 차이가 나타남
사후분석: pval의 값이 0.05보다 낮으므로, 방송프로그램과 연예인 간의 차이가 유의하여 주효과가 나타난다.

### 5. 시계열 / 워드 클라우드(자주 사용하는 단어 찾기) 47- 하니칼국수 - 성시경 / 181-코카모메-츄릅켠 / 270-전참시-몽탄

In [ ]:
lists = ["하니칼국수_성시경", "코카모메_츄릅켠", "몽탄_전참시"]
for item in lists:
    file = "./data/naver_blog/rowdata/" + item + ".csv"
    temp = pd.read_csv(file)
    temp.sort_values(by = "postdate", ascending=True, inplace=True)
    temp.postdate = pd.to_datetime(temp['postdate'])
    # 기간 나누기
    # temp['period'] = temp['postdate'].dt.to_period(freq='M')  # 년-월
    # 그래프 그리기
    plt.figure(figsize=(10,6))
    temp["test"] = 1
    plt.hist(temp["postdate"], weights=temp["test"], bins=10, cumulative = True, color="orange")
    plt.hist(temp["postdate"], weights=temp["test"], bins=10, cumulative = False, color="skyblue")
    plt.title(item + " 시계열 그래프")
    plt.show()

In [ ]:
filelists = ["하니칼국수_성시경", "코카모메_츄릅켠", "몽탄_전참시"]
referenceDate = ['2021-05', '2021-08', "2021-03"]
for i in range(len(filelists)):
    file = "./data/naver_blog/rowdata/" + filelist[i] + ".csv"
    temp = pd.read_csv(file)
    temp.sort_values(by = "postdate", ascending=True, inplace=True)
    temp.postdate = pd.to_datetime(temp['postdate'])
    # 기간 나누기
    temp['period'] = temp['postdate'].dt.to_period(freq='M')  # 년-월
    # 기간별로 포스트 개수 세기
    period_count = temp.groupby("period").count()[["postdate"]]
    period_count.rename(columns = {"postdate": "count"}, inplace=True)
    # 누적합 열 추가하기
    period_count["cum"] = period_count[["count"]].cumsum()
    # 그래프 그리기
    plt.figure(figsize=(16,8))
    # 그래프를 그리기 위한 형변환
    period_count.index=period_count.index.to_series().astype(str)
    # 기준일 색깔 넣기
    my_color = np.where(period_count.index == referenceDate[0], 'skyblue', 'orange')
    # 라인그래프 그리기
    plt.plot(period_count.index, period_count["cum"], marker='o')
    # 기준일 그리기
    plt.axvline(referenceDate[i], color='lightgray', linestyle='--', linewidth=2)
    plt.text(referenceDate[i], period_count["cum"].max() * 0.95, "기준선",       
             fontsize = 15, 
             color='lightgray',
             horizontalalignment='center') 
    # 바그래프 그리기
    plt.bar(period_count.index, period_count["count"], color="blue",align='center')
    plt.title(filelists[i] + " 시계열 그래프")
    plt.savefig('./output/' + filelists[i] + ' 시계열 그래프.png')
    plt.show()

# 문제모음

1. 인스턴스가 삭제가 안됩니다.
2. 파일 데이터를 불러올때 몇몇의 파일을 읽어오지 못합니다.
3. seaborn에서 한글 폰트를 인식하지 못합니다.

In [ ]:
# 폰트설정2
sns.set(font="Malgun Gothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

# 폰트설정3
jupyter notebook 내 그래프를 바로 그리기 위한 설정
%matplotlib inline
# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

# 폰트설정4
%matplotlib inline
import platform
path = './data/THEdog.ttf'
from matplotlib import font_manager, rc

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unkonwn system... sorry~~~~')